<a href="https://colab.research.google.com/github/Deepikadhinakaran/AI_price_optima/blob/main/fast_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
!pip install fastapi uvicorn pyngrok nest_asyncio joblib


In [74]:
import os
import io
import joblib
import nest_asyncio
import pandas as pd
import numpy as np
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pyngrok import ngrok


In [75]:
MODEL_PATH = "gradient_boosting_model.pkl"
DATA_CSV   = "dynamic_pricing.csv"

try:
    df_base_all = pd.read_csv(DATA_CSV)
except Exception as e:
    raise RuntimeError(f"❌ Could not load {DATA_CSV}: {e}")

try:
    best_pipe = joblib.load(MODEL_PATH)
except Exception as e:
    raise RuntimeError(f"❌ Could not load model {MODEL_PATH}: {e}")


In [76]:

app = FastAPI(title="Dynamic Pricing API", version="1.0.0")

#Enable CORS (so frontend apps can connect)

app.add_middleware(
CORSMiddleware,
allow_origins=[""],
allow_credentials=True,
allow_methods=[""],
allow_headers=["*"],
)

In [77]:
class RecommendRequest(BaseModel):
    record: dict

In [78]:
@app.get("/health")
def health():
    return {"ok": True, "rows_loaded": int(len(df_base_all))}

@app.post("/recommend")
def recommend(req: RecommendRequest):
    row = pd.DataFrame([req.record])

    # Add missing columns if needed
    missing = [c for c in df_base_all.columns if c not in row.columns]
    for m in missing:
        row[m] = 0

    try:
        pred = best_pipe.predict(row.drop("Price", axis=1, errors="ignore"))[0]
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

    return {"recommended_price": float(pred)}


@app.post("/recommend_batch")
def recommend_batch(file: UploadFile = File(...)):
    try:
        df = pd.read_csv(file.file)
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Invalid file: {e}")

    try:
        preds = best_pipe.predict(df.drop("Price", axis=1, errors="ignore"))
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

    df["recommended_price"] = preds
    return {"rows": len(df), "sample": df.head(3).to_dict(orient="records")}

In [79]:
import uvicorn

In [80]:
nest_asyncio.apply()

In [81]:
!pip install pyngrok

In [82]:
import os
from pyngrok import ngrok


In [83]:
os.environ["NGROK_AUTH_TOKEN"] = "YOUR_NGROK_AUTH_TOKEN"

In [84]:
%%writefile fast.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"Hello": "World"}


Overwriting fast.py


In [ ]:
!uvicorn fast:app --host 0.0.0.0 --port 5000
public_url = ngrok.connect(5000)
print("🚀 Public API URL:", public_url)


INFO:     Started server process [30391]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)
